Creates `.csv` files for UTD to run `ProgressCaptioner` on it.

In [1]:
import os
import sys
sys.path.append("/athenahomes/piyush/projects/ProgCaptioner/")

import pandas as pd
import numpy as np

import shared.utils as su

Failed to import torchvideotransforms. Proceeding without.
Please install using:
pip install git+https://github.com/hassony2/torch_videovision


In [12]:
data_dir = "/scratch/shared/beegfs/piyush/datasets/UTD/UTD_dataset"

split_file = os.path.join(data_dir, "UTD_splits/splits_msrvtt_test.json")
video_ids = su.io.load_json(split_file)['debiased']

# Load data
data = su.io.load_json(
    os.path.join(data_dir, "UTD_descriptions/descriptions_msrvtt_test.json")
)
data = {k: data[k] for k in video_ids}

len(video_ids), len(data)

(718, 718)

In [15]:
data['video7020'].keys()

dict_keys(['objects+composition+activities', 'objects', 'activities', 'verbs', 'objects+composition+activities_15_words'])

In [16]:
data['video7020']['objects+composition+activities_15_words']

['Person decorating a round, frosted cake with a green leaf, held in spread fingers, atop a grid-patterned surface.',
 'Person shapes green clay or dough on a reflective paper or plastic surface. Hands are actively manipulating and creating.',
 "Person's hand shapes or picks up small, green, leaf-shaped craft item on textured red-blue surface. Creative activity implied.",
 'Person holds small, green, leaf-shaped object, possibly plastic, over patterned surface, delicately handled in artistic or crafting scene.',
 'Two hands with spread fingers and palms down on a grid for measuring or calibrating, with symmetrical placement and aligned fingers, on a neutral, light-background.',
 'Two hands carefully place a flower-shaped white object into a grid with red lines, using a brush-like tool for application or spreading substance. Craft or baking project in progress.',
 'Two hands shaping a yellow, flower-design object on a grid for a crafting project, with a related tool or material in the b

In [34]:
data_dir = "/scratch/shared/beegfs/piyush/datasets/MSRVTT"

data = su.io.load_json(
    os.path.join(data_dir, "annotation/MSR_VTT.json")
)['annotations']
df_caption = pd.DataFrame(data)

df = df_caption[['image_id']].drop_duplicates()
df["id"] = df.image_id.apply(lambda x: os.path.join("all", x))

df

,image_id,id
0,video2960,all/video2960
20,video2636,all/video2636
40,video4311,all/video4311
60,video1844,all/video1844
80,video2213,all/video2213
...,...,...
199894,video7795,all/video7795
199914,video7112,all/video7112
199934,video8658,all/video8658
199954,video8978,all/video8978


,caption,id,image_id
0,a cartoon animals runs through an ice cave in ...,0,video2960
1,a cartoon character runs around inside of a vi...,1,video2960
2,a character is running in the snow,2,video2960
3,a person plays a video game centered around ic...,3,video2960
4,a person plays online and records themselves,4,video2960
...,...,...,...
199989,shows a man in a red sweeter and white shirt ...,199995,video8600
199990,a man explains how to save money using careful...,199996,video8600
199991,a person with maroon tshirt speaks in the news...,199997,video8600
199992,the man in a purple sweater is giving a news s...,199998,video8600


In [29]:
video_ids = np.unique(np.array([x['id'] for x in data['images']]))
len(video_ids)

10000

In [18]:
data.keys()

dict_keys(['info', 'images', 'licenses', 'type', 'annotations'])

In [36]:
import transformers
transformers.__version__

'4.40.0.dev0'

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Exception: data did not match any variant of untagged enum ModelWrapper at line 757479 column 3